# Chapter 19: Training and Deploying TensorFlow Models at Scale

Pada chapter ini, pembahasan Machine Learning bergeser dari **membangun model** ke **mengoperasikan model di dunia nyata**.

Jika pada chapter-chapter sebelumnya fokus utama adalah bagaimana melatih model agar akurat, maka chapter ini menjawab pertanyaan penting berikut:

**"Apa yang harus dilakukan setelah model selesai dilatih?"**

Chapter ini membahas bagaimana sebuah model TensorFlow dapat:
- dideploy ke lingkungan produksi,
- digunakan oleh berbagai sistem lain melalui API,
- diperbarui secara berkala tanpa mengganggu layanan,
- diskalakan untuk menangani trafik tinggi,
- serta dipercepat menggunakan hardware seperti GPU dan TPU.

Topik-topik ini sangat krusial karena pada praktiknya, nilai bisnis dari Machine Learning baru benar-benar terasa ketika model berhasil digunakan secara **andal, efisien, dan berkelanjutan** di production environment.

## 1. Dari Model Eksperimental ke Model Produksi

Melatih model di notebook atau script Python hanyalah **langkah awal**.

Dalam skenario nyata, sebuah sistem Machine Learning biasanya harus:
- menerima data secara real-time,
- memberikan prediksi dengan latensi rendah,
- melayani banyak permintaan secara bersamaan,
- diperbarui ketika data baru tersedia.

Model yang akurat tetapi tidak dapat diintegrasikan dengan sistem lain **tidak memiliki nilai praktis**.

Sebagai contoh:
- sistem rekomendasi berita harus terus diperbarui mengikuti tren terbaru,
- model fraud detection harus merespons transaksi secara real-time,
- model prediksi harus dapat digunakan oleh banyak aplikasi sekaligus.

Kondisi ini menuntut pendekatan yang lebih sistematis terhadap **deployment, versioning, dan scalability**.

## 2. Model Serving sebagai Layanan

Pendekatan umum dalam production Machine Learning adalah **memisahkan model dari aplikasi utama**.

Alih-alih menjalankan model langsung di dalam setiap aplikasi, model dibungkus sebagai sebuah **layanan (service)** yang dapat diakses melalui API.

Pendekatan ini memberikan banyak keuntungan:
- aplikasi lain tidak perlu mengetahui detail implementasi model,
- model dapat diganti atau diperbarui tanpa mengubah kode aplikasi,
- sistem dapat diskalakan secara independen.

Model serving biasanya dilakukan menggunakan:
- REST API (berbasis HTTP dan JSON),
- atau gRPC (berbasis protocol buffer, lebih efisien).

Dengan cara ini, model menjadi **komponen terpisah** dalam arsitektur sistem, mirip dengan microservice.

## 3. Tantangan Production Machine Learning

Mendeploy model ke production membawa tantangan baru yang tidak muncul saat eksperimen:

- **Model Versioning**: bagaimana mengganti model lama dengan model baru tanpa downtime,
- **Rollback**: bagaimana kembali ke model sebelumnya jika versi baru bermasalah,
- **A/B Testing**: bagaimana menguji dua model secara paralel,
- **Scalability**: bagaimana melayani ribuan hingga jutaan request per detik,
- **Monitoring**: bagaimana memantau performa dan degradasi model.

Chapter ini akan membahas solusi praktis terhadap tantangan-tantangan tersebut menggunakan ekosistem TensorFlow.

## 4. Gambaran Umum Chapter 19

Secara garis besar, chapter ini akan membahas:

- penyimpanan model dalam format **SavedModel**,
- deployment menggunakan **TensorFlow Serving**,
- deployment ke cloud menggunakan **Google Cloud AI Platform**,
- deployment ke mobile dan embedded device menggunakan **TensorFlow Lite**,
- deployment ke browser menggunakan **TensorFlow.js**,
- percepatan komputasi dengan **GPU**,
- serta training terdistribusi menggunakan strategi TensorFlow.

Topik-topik ini membentuk fondasi penting bagi sistem Machine Learning skala industri.

## 5. Saving and Exporting Models

Sebelum sebuah model dapat dideploy ke production, model tersebut harus disimpan dalam format yang **standar, stabil, dan dapat digunakan lintas platform**.

TensorFlow menyediakan format penyimpanan resmi bernama **SavedModel**, yang dirancang khusus untuk kebutuhan deployment skala besar.

### 5.1 Mengapa Tidak Menyimpan Model sebagai File Python?

Menyimpan model hanya sebagai kode Python tidak cukup untuk deployment karena:
- bergantung pada environment Python tertentu,
- sulit digunakan oleh bahasa atau sistem lain,
- tidak menyimpan graph komputasi secara eksplisit.

Production environment membutuhkan format yang **language-agnostic** dan **robust**.

## 5.2 SavedModel Format

**SavedModel** adalah format standar TensorFlow untuk menyimpan:
- arsitektur model,
- bobot (weights),
- computation graph,
- signature fungsi untuk inference.

Format ini dapat digunakan oleh:
- TensorFlow Serving,
- TensorFlow Lite,
- TensorFlow.js,
- serta berbagai runtime TensorFlow lainnya.

SavedModel disimpan sebagai sebuah **direktori**, bukan satu file tunggal.

Di dalam direktori tersebut biasanya terdapat:
- folder `assets/` (opsional),
- folder `variables/` (berisi bobot model),
- file `saved_model.pb` (graph dan metadata model).

### 5.3 Menyimpan Model dengan Keras

Keras menyediakan method `save()` untuk menyimpan model dalam format SavedModel.

Contoh berikut menunjukkan cara menyimpan model hasil training.

In [ ]:
model.save("my_saved_model")

Perintah di atas akan membuat sebuah direktori bernama `my_saved_model/`.

Direktori ini dapat langsung digunakan untuk deployment tanpa perlu kode training tambahan.

### 5.4 Memuat Kembali Model

Model yang telah disimpan dapat dimuat kembali menggunakan method `load_model()`.

In [ ]:
loaded_model = tf.keras.models.load_model("my_saved_model")

Model yang dimuat kembali akan memiliki:
- arsitektur yang sama,
- bobot yang sama,
- serta siap digunakan untuk inference.

Hal ini memungkinkan pemisahan yang jelas antara fase **training** dan **serving**.

### 5.5 Model Signatures

SavedModel juga menyimpan **signature**, yaitu definisi fungsi input dan output yang dapat dipanggil oleh sistem lain.

Signature ini sangat penting untuk:
- TensorFlow Serving,
- pemanggilan model via REST atau gRPC,
- integrasi dengan sistem non-Python.

Dengan adanya signature, sistem eksternal tidak perlu mengetahui detail internal model.

Mereka hanya perlu mengetahui:
- nama fungsi,
- format input,
- format output.

## 6. TensorFlow Serving

**TensorFlow Serving** adalah sistem khusus yang dirancang untuk **menyajikan model TensorFlow di lingkungan produksi**.

Tujuan utama TensorFlow Serving adalah:
- menyediakan inference dengan latensi rendah,
- mendukung high throughput,
- memungkinkan versioning dan hot-swapping model,
- terintegrasi dengan format SavedModel.

TensorFlow Serving memisahkan:
- **training environment** (eksperimen, notebook, GPU training),
- **serving environment** (server inference yang stabil dan efisien).

Dengan pemisahan ini, sistem menjadi lebih modular dan mudah dikelola.

## 6.1 Arsitektur TensorFlow Serving

TensorFlow Serving berjalan sebagai **server mandiri** yang:
- memuat model dari disk,
- mengekspos endpoint API,
- menangani request inference dari berbagai klien.

Server ini tidak bergantung pada kode Python training.

TensorFlow Serving mendukung dua protokol utama:
- **REST API** (berbasis HTTP + JSON),
- **gRPC** (berbasis Protocol Buffers, lebih cepat dan efisien).

Pemilihan protokol tergantung pada kebutuhan performa dan kompleksitas sistem.

## 6.2 Menjalankan TensorFlow Serving dengan Docker

Cara paling praktis untuk menjalankan TensorFlow Serving adalah menggunakan **Docker image resmi**.

Docker memastikan:
- environment konsisten,
- dependensi terkelola,
- deployment yang mudah direplikasi.

In [ ]:
docker run -p 8501:8501 \
  --mount type=bind,source=/path/to/my_saved_model,target=/models/my_model \
  -e MODEL_NAME=my_model \
  tensorflow/serving

Perintah di atas akan:
- menjalankan TensorFlow Serving,
- memuat model dari direktori lokal,
- mengekspos REST API pada port 8501.

Model dapat diakses tanpa menjalankan kode Python sama sekali.

## 6.3 REST API untuk Inference

Dengan REST API, inference dilakukan melalui HTTP POST request menggunakan format JSON.

Endpoint standar TensorFlow Serving berbentuk:

`http://localhost:8501/v1/models/<model_name>:predict`

In [ ]:
import requests
import json

data = json.dumps({
    "instances": X_test[:1].tolist()
})

response = requests.post(
    "http://localhost:8501/v1/models/my_model:predict",
    data=data
)

response.json()

Response dari server berisi hasil prediksi model dalam format JSON.

Pendekatan ini memungkinkan berbagai bahasa pemrograman mengakses model yang sama.

## 6.4 gRPC API

Selain REST, TensorFlow Serving juga mendukung **gRPC**, yang:
- lebih efisien dibandingkan JSON,
- cocok untuk sistem dengan throughput tinggi,
- sering digunakan pada arsitektur microservices berskala besar.

Namun, gRPC lebih kompleks dalam implementasi dibandingkan REST.

## 7. Model Versioning

Dalam lingkungan produksi, model Machine Learning **tidak bersifat statis**.

Seiring waktu:
- data baru tersedia,
- distribusi data dapat berubah (data drift),
- model lama dapat mengalami penurunan performa.

Karena itu, sistem produksi harus mendukung **model versioning**, yaitu kemampuan menyimpan dan mengelola beberapa versi model secara bersamaan.

### 7.1 Versioning pada TensorFlow Serving

TensorFlow Serving secara alami mendukung versioning melalui struktur direktori.

Setiap versi model disimpan dalam subdirektori bernomor, misalnya:

```
my_model/
 ├── 1/
 │    └── saved_model.pb
 ├── 2/
 │    └── saved_model.pb
```

Server akan otomatis memuat **versi terbaru** berdasarkan nomor versi.

Pendekatan ini memungkinkan:
- update model tanpa downtime,
- penyimpanan beberapa versi secara paralel,
- kemudahan rollback jika terjadi masalah.

## 8. A/B Testing untuk Model

**A/B testing** adalah teknik untuk membandingkan dua atau lebih versi model secara bersamaan di lingkungan produksi.

Tujuannya adalah mengevaluasi model baru secara **nyata** menggunakan data pengguna sebenarnya, bukan hanya data offline.

Dalam A/B testing model:
- sebagian request diarahkan ke model lama (baseline),
- sebagian request diarahkan ke model baru (candidate),
- metrik performa dibandingkan secara statistik.

Pendekatan ini membantu memastikan bahwa model baru benar-benar lebih baik sebelum diterapkan sepenuhnya.

TensorFlow Serving dapat dikombinasikan dengan:
- load balancer,
- API gateway,
- atau sistem routing eksternal,

untuk mengatur distribusi trafik ke versi model yang berbeda.

## 9. Rollback Strategy

Tidak semua deployment model berjalan mulus.

Model baru bisa saja:
- menghasilkan prediksi yang tidak stabil,
- meningkatkan latensi,
- atau menurunkan performa bisnis.

Karena itu, sistem produksi harus memiliki **strategi rollback** yang cepat dan aman.

Dengan versioning di TensorFlow Serving, rollback dapat dilakukan dengan:
- menghentikan penggunaan versi terbaru,
- mengarahkan kembali trafik ke versi sebelumnya,
- tanpa perlu retraining atau rebuild sistem.

Strategi ini sangat penting untuk menjaga **reliability** sistem Machine Learning di production.

## 10. Cloud Deployment

Untuk sistem Machine Learning berskala besar, deployment lokal sering kali tidak cukup.

Cloud computing memungkinkan:
- skalabilitas otomatis,
- ketersediaan tinggi (high availability),
- integrasi dengan berbagai layanan data dan monitoring.

Pada chapter ini, contoh cloud deployment difokuskan pada **Google Cloud AI Platform** (sekarang dikenal sebagai Vertex AI).

## 10.1 Mengapa Menggunakan Cloud untuk ML?

Beberapa alasan utama penggunaan cloud dalam deployment Machine Learning:
- model dapat diakses dari mana saja,
- sistem dapat menangani lonjakan trafik secara otomatis,
- tidak perlu mengelola server fisik sendiri,
- mudah terintegrasi dengan pipeline data dan monitoring.

Pendekatan ini sangat umum pada sistem ML di industri.

## 10.2 AI Platform / Vertex AI Overview

Google Cloud AI Platform menyediakan layanan end-to-end untuk Machine Learning, mulai dari:
- training model,
- hyperparameter tuning,
- deployment model,
- hingga monitoring performa model.

Platform ini dirancang agar model TensorFlow dapat dideploy tanpa perlu membangun sistem serving dari nol.

## 10.3 Menyiapkan Model untuk Cloud Deployment

Sama seperti TensorFlow Serving, model yang akan dideploy ke cloud harus disimpan dalam format **SavedModel**.

Format ini memastikan kompatibilitas dengan layanan managed serving di cloud.

Model kemudian diunggah ke **Cloud Storage (GCS)**, yang berfungsi sebagai repositori model terpusat.

Setiap versi model biasanya disimpan pada path yang berbeda untuk mendukung versioning.

In [ ]:
# contoh struktur path di Google Cloud Storage
# gs://my-bucket/my_model/1/
# gs://my-bucket/my_model/2/

## 10.4 Deploying Model to AI Platform

Setelah model berada di Cloud Storage, model dapat dideploy sebagai **endpoint**.

Endpoint ini akan:
- memuat model secara otomatis,
- menangani request inference,
- menskalakan resource sesuai kebutuhan.

Deployment biasanya dilakukan menggunakan:
- Google Cloud Console,
- atau command-line tool `gcloud`.

Pendekatan ini memisahkan proses deployment dari kode aplikasi.

## 10.5 Keuntungan Managed ML Services

Dengan menggunakan layanan managed seperti AI Platform, tim ML mendapatkan beberapa keuntungan:
- tidak perlu mengelola server inference,
- autoscaling otomatis,
- integrasi keamanan dan logging bawaan,
- fokus pada pengembangan model, bukan infrastruktur.

Namun, pendekatan ini juga memiliki trade-off seperti biaya dan ketergantungan pada vendor cloud tertentu.

## 11. Deployment ke Mobile & Embedded Devices

Tidak semua model Machine Learning dijalankan di server cloud.

Dalam banyak aplikasi nyata, model perlu dijalankan langsung di **perangkat pengguna**, seperti:
- smartphone,
- IoT devices,
- embedded systems,
- edge devices.

Untuk kebutuhan ini, TensorFlow menyediakan **TensorFlow Lite (TFLite)**.

## 11.1 Mengapa On-Device ML?

Menjalankan model langsung di perangkat memiliki beberapa keuntungan:
- latensi sangat rendah (tidak perlu request ke server),
- privasi data lebih terjaga,
- aplikasi tetap berfungsi tanpa koneksi internet,
- beban server dan biaya cloud berkurang.

Namun, perangkat memiliki keterbatasan memori, daya, dan komputasi.

## 11.2 TensorFlow Lite

**TensorFlow Lite** adalah versi ringan dari TensorFlow yang dirancang khusus untuk perangkat dengan resource terbatas.

TFLite menyediakan:
- format model yang lebih kecil,
- runtime inference yang cepat,
- dukungan hardware acceleration (CPU, GPU, DSP, NPU).

Model TFLite biasanya dihasilkan dari model TensorFlow standar melalui proses konversi.

## 11.3 Mengonversi Model ke TensorFlow Lite

Model yang telah disimpan dalam format SavedModel dapat dikonversi ke format TFLite menggunakan **TFLiteConverter**.

Proses ini menghasilkan file `.tflite` yang siap digunakan di perangkat mobile atau embedded.

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("my_saved_model")
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)

File `model.tflite` yang dihasilkan memiliki ukuran jauh lebih kecil dibandingkan model TensorFlow penuh.

File ini dapat langsung disertakan dalam aplikasi Android, iOS, atau firmware embedded.

## 11.4 Model Optimization

Untuk meningkatkan performa dan mengurangi ukuran model, TensorFlow Lite mendukung berbagai teknik optimisasi, seperti:
- **quantization** (mengubah bobot dari float32 ke int8),
- pruning,
- operator fusion.

Optimisasi ini penting untuk deployment pada perangkat dengan memori dan daya terbatas.

Quantization, misalnya, dapat:
- memperkecil ukuran model secara signifikan,
- mempercepat inference,
- dengan penurunan akurasi yang minimal jika dilakukan dengan benar.

## 11.5 Use Cases TensorFlow Lite

Beberapa contoh penggunaan TensorFlow Lite:
- face detection di smartphone,
- speech recognition offline,
- image classification pada IoT camera,
- sensor data analysis pada embedded device.

Pendekatan ini dikenal sebagai **edge ML**, dan semakin penting seiring berkembangnya IoT dan mobile computing.

## 12. Deployment di Browser dengan TensorFlow.js

Selain server dan perangkat mobile, model Machine Learning juga dapat dijalankan langsung di **browser pengguna**.

Pendekatan ini dimungkinkan melalui **TensorFlow.js**, sebuah library JavaScript yang memungkinkan:
- inference model ML langsung di browser,
- training model sederhana di sisi klien,
- pemanfaatan GPU melalui WebGL atau WebGPU.

Deployment di browser membuka peluang besar untuk aplikasi interaktif dan real-time tanpa server inference.

## 12.1 Keuntungan Deployment di Browser

Menjalankan model di browser memiliki beberapa kelebihan:
- tidak memerlukan backend inference,
- latensi sangat rendah,
- privasi data lebih terjaga karena data tidak dikirim ke server,
- mudah diakses melalui web.

Namun, pendekatan ini juga dibatasi oleh kemampuan hardware perangkat pengguna.

## 12.2 TensorFlow.js Overview

**TensorFlow.js** memungkinkan pengembang untuk:
- memuat model TensorFlow atau Keras di JavaScript,
- menjalankan inference menggunakan CPU atau GPU browser,
- membangun aplikasi ML end-to-end berbasis web.

TensorFlow.js mendukung model hasil konversi dari SavedModel atau Keras.

## 12.3 Mengonversi Model ke TensorFlow.js

Model TensorFlow dapat dikonversi ke format TensorFlow.js menggunakan tool `tensorflowjs_converter`.

Tool ini akan mengubah model menjadi file JSON dan binary weights yang dapat dimuat di browser.

In [ ]:
tensorflowjs_converter \
  --input_format=tf_saved_model \
  --output_format=tfjs_graph_model \
  my_saved_model \
  tfjs_model

Setelah konversi, direktori `tfjs_model/` akan berisi:
- file model.json,
- file binary weights.

File-file ini dapat disajikan menggunakan web server biasa.

## 12.4 Menggunakan Model di JavaScript

Model TensorFlow.js dapat dimuat dan digunakan langsung di kode JavaScript.

In [ ]:
// Contoh JavaScript (konseptual)
const model = await tf.loadGraphModel('tfjs_model/model.json');
const prediction = model.predict(inputTensor);


Pendekatan ini memungkinkan integrasi Machine Learning langsung ke aplikasi web modern.

TensorFlow.js banyak digunakan untuk aplikasi seperti:
- gesture recognition,
- image classification berbasis webcam,
- interactive data visualization.

## 13. Hardware Acceleration

Seiring bertambahnya ukuran dataset dan kompleksitas model, training dan inference pada CPU saja sering kali menjadi tidak efisien.

Untuk mengatasi hal ini, TensorFlow mendukung **hardware acceleration** menggunakan perangkat khusus seperti:
- **GPU (Graphics Processing Unit)**,
- **TPU (Tensor Processing Unit)**.

Perangkat ini dirancang untuk melakukan komputasi paralel dalam jumlah besar, yang sangat cocok untuk operasi linear algebra pada neural network.

## 13.1 GPU Acceleration

GPU sangat efektif untuk neural network karena:
- mampu menjalankan ribuan operasi paralel,
- sangat efisien untuk matrix multiplication,
- didukung luas oleh framework deep learning modern.

TensorFlow dapat secara otomatis mendeteksi dan menggunakan GPU jika tersedia.

Dengan GPU, proses training yang sebelumnya memakan waktu berjam-jam di CPU dapat dipercepat menjadi hitungan menit.

Namun, penggunaan GPU juga membutuhkan:
- driver dan library khusus (CUDA, cuDNN),
- manajemen memori yang lebih kompleks.

## 13.2 TPU (Tensor Processing Unit)

**TPU** adalah hardware khusus yang dikembangkan oleh Google untuk mempercepat workload Machine Learning.

TPU dirancang untuk:
- operasi tensor skala besar,
- training dan inference neural network secara efisien,
- integrasi mendalam dengan TensorFlow.

TPU umumnya tersedia melalui layanan cloud seperti Google Cloud.

Penggunaan TPU memungkinkan:
- training model besar dalam waktu yang jauh lebih singkat,
- skala training yang sulit dicapai dengan satu GPU.

Namun, TPU memiliki keterbatasan:
- hanya mendukung operasi tertentu,
- kurang fleksibel dibandingkan GPU untuk eksperimen.

## 14. Distributed Training

Untuk dataset dan model yang sangat besar, satu perangkat komputasi saja sering kali tidak cukup.

**Distributed training** memungkinkan training model dilakukan secara paralel di:
- beberapa GPU,
- beberapa mesin,
- atau kombinasi keduanya.

TensorFlow menyediakan **tf.distribute.Strategy** untuk mempermudah distributed training.

Beberapa strategi yang umum digunakan:
- `MirroredStrategy`: training paralel pada banyak GPU di satu mesin,
- `MultiWorkerMirroredStrategy`: training pada banyak mesin,
- `TPUStrategy`: training pada TPU.

### 14.1 Contoh Penggunaan MirroredStrategy

Contoh berikut menunjukkan bagaimana model Keras dapat dilatih menggunakan beberapa GPU secara transparan.

In [ ]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=[28, 28]),
        keras.layers.Dense(300, activation="relu"),
        keras.layers.Dense(100, activation="relu"),
        keras.layers.Dense(10, activation="softmax")
    ])
    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer="sgd",
        metrics=["accuracy"]
    )

Dengan pendekatan ini:
- kode training hampir tidak berubah,
- TensorFlow mengelola sinkronisasi gradien secara otomatis,
- performa training meningkat secara signifikan.

Distributed training merupakan komponen penting dalam sistem Machine Learning berskala industri.

## Closing Summary (Chapter 19)

Chapter 19 membahas aspek yang sangat krusial dalam Machine Learning modern, yaitu **bagaimana membawa model dari tahap eksperimen ke lingkungan produksi berskala besar**.

Jika chapter sebelumnya berfokus pada bagaimana membangun dan melatih model yang akurat, maka chapter ini menekankan bagaimana model tersebut dapat digunakan secara nyata, andal, dan berkelanjutan.

Beberapa poin utama yang dipelajari dalam chapter ini meliputi:
- penyimpanan model menggunakan format **SavedModel** sebagai standar deployment,
- penyajian model menggunakan **TensorFlow Serving** dengan REST dan gRPC,
- pengelolaan **model versioning**, A/B testing, dan rollback,
- deployment model ke cloud menggunakan layanan managed seperti **Google Cloud AI Platform**,
- deployment ke perangkat mobile dan embedded menggunakan **TensorFlow Lite**,
- deployment ke browser menggunakan **TensorFlow.js**,
- percepatan komputasi dengan **GPU** dan **TPU**,
- serta training terdistribusi menggunakan strategi TensorFlow.

Chapter ini menegaskan bahwa keberhasilan sistem Machine Learning di dunia nyata tidak hanya ditentukan oleh akurasi model, tetapi juga oleh:
- stabilitas sistem,
- kemampuan untuk diskalakan,
- kemudahan pemeliharaan,
- serta kesiapan menghadapi perubahan data dan kebutuhan bisnis.

Dengan kata lain, **Machine Learning di industri adalah kombinasi antara data science dan software engineering**.

Pemahaman chapter ini sangat penting bagi peran seperti:
- Machine Learning Engineer,
- Data Engineer,
- AI Engineer,
- dan praktisi ML yang ingin membangun sistem end-to-end.

Chapter 19 menjadi penutup yang kuat untuk menunjukkan bagaimana TensorFlow tidak hanya digunakan untuk eksperimen, tetapi juga untuk membangun sistem Machine Learning **siap produksi dan skala industri**.